In [1]:
import math
import warnings
tr.fit(d)
print(tr.quantiles_)
max_share = 0.5
n_quantiles = tr.n_quantiles_
col_idx = 0

max_n_quantiles_per_category = math.floor(n_quantiles * max_share)
if max_n_quantiles_per_category == 0:
    warnings.warn(f"You have chose a max_share={max_share} along with n_quantiles={n_quantiles} such that the maximum number of categories per unique value is less than 1. This is automatically changed such that maximum number of categories per unique value is equal to 1.")
    max_n_quantiles_per_category = 1
col_unique_values, col_value_counts = np.unique(tr.quantiles_[:, col_idx], return_counts=True)
percentages = col_value_counts / sum(col_value_counts)
if max(percentages) > max_share:
    indices_too_common_values = np.where(percentages > max_share)[0]
    print("Too common values:", col_unique_values[indices_too_common_values])
    # Freeing up available quantile spaces from values that are too common
    # (have a larger share than permitted by the parameter max_share)
    col_bool_remaining_values = np.ones(d["a"].shape[0], dtype=bool)
    for curr_common_value_idx in indices_too_common_values:
        curr_common_value = col_unique_values[curr_common_value_idx]
        curr_common_value_count = col_value_counts[curr_common_value_idx]
        n_new_avail_quantiles = curr_common_value_count - max_n_quantiles_per_category
        print("curr_common_value:", curr_common_value, "n_new_avail_quantiles", n_new_avail_quantiles)
        curr_avail_quantile_indices = np.nonzero(np.isclose(tr.quantiles_, curr_common_value))[0][-n_new_avail_quantiles:]
        tr.quantiles_[curr_avail_quantile_indices, col_idx] = np.nan
        col_bool_remaining_values *= np.logical_not(np.isclose(d["a"], curr_common_value))
        
    
    # Reset the quantile spaces for values with shares smaller than the parameter max_share, before a redraw
    col_remaining_values = d["a"].loc[np.logical_not(np.isclose(d["a"],  most_common_value))].to_numpy()
    indices_less_common = np.where(percentages <= max_share)
    values_less_common = col_unique_values[indices_less_common]
    for curr_less_common_value in values_less_common:
        print("curr_less_common_value", curr_less_common_value)
        tr.quantiles_[np.nonzero(np.isclose(tr.quantiles_, curr_less_common_value))[0], col_idx] = np.nan
    
    print(tr.quantiles_)
    
    available_quantile_indices = np.where(np.isnan(tr.quantiles_))[0]
    n_available_quantiles = len(available_quantile_indices)
    print("Available quantile indices", available_quantile_indices)
    print("n_available_quantiles:", n_available_quantiles)
    
    
    print(col_bool_remaining_values)
    col_remaining_values = d["a"].loc[col_bool_remaining_values].to_numpy()
    print(col_remaining_values)
    if col_remaining_values.shape[0] != n_available_quantiles:
        print("less than")
        # Create empty vector for storing remaining quantiles (both from dataset and interpolated)
        temp_new_less_common = np.repeat(np.nan, n_available_quantiles)
        
        # Initiate remaining dataset values at random places in the empty vector, but with values sorted in ascending order
        temp_indices = np.arange(1, n_available_quantiles, dtype=int)
        temp_shuffled_indices = np.random.choice(temp_indices, size=temp_indices.shape[0], replace=False)
        temp_not_nan_indices = np.sort(temp_shuffled_indices[:col_remaining_values.shape[0]])
        temp_nan_indices = temp_shuffled_indices[col_remaining_values.shape[0]:]
        
        temp_new_less_common[temp_not_nan_indices] = np.sort(col_remaining_values)
        # Initiate the remaining places in the vector with linearly interpolated values based on the
        # randomly placed (but sorted) values from the dataset.
        temp_new_less_common[temp_nan_indices] = np.interp(temp_nan_indices,
                                                           temp_not_nan_indices,
                                                           temp_new_less_common[temp_not_nan_indices]
                                                          )
        print("temp_new_less_common")
        tr.quantiles_[available_quantile_indices, col_idx] = temp_new_less_common
    elif col_remaining_values.shape[0] == n_available_quantiles:
        print("equal")
        tr.quantiles_[available_quantile_indices, col_idx] = col_remaining_values
    else:
        print("more than")
        tr.quantiles_[available_quantile_indices, col_idx] = np.random.choice(col_remaining_values,
                                                                              size=n_available_quantiles, replace=False)
    tr.quantiles_[:, col_idx] = np.sort(tr.quantiles_[:, col_idx])

print(tr.quantiles_)

NameError: name 'tr' is not defined

In [ ]:
import math
import warnings
tr.fit(d)
print(tr.quantiles_)
max_share = 0.6
n_quantiles = tr.n_quantiles_
col_idx = 0

max_n_quantiles_per_value = math.floor(n_quantiles * max_share)
print("max quantiles", max_n_quantiles_per_value)
if max_n_quantiles_per_value == 0:
    warnings.warn(f"You have chose a max_share={max_share} along with n_quantiles={n_quantiles} such that the maximum number of categories per unique value is less than 1. This is automatically changed such that maximum number of categories per unique value is equal to 1.")
    max_n_quantiles_per_value = 1
col_unique_values, col_value_counts = np.unique(tr.quantiles_[:, col_idx], return_counts=True)
col_percentages = col_value_counts / sum(col_value_counts)
print("col_percentages", col_percentages)
if max(col_percentages) > max_share:
    indices_too_common_values = np.where(col_percentages > max_share)[0]
    print("Too common values:", col_unique_values[indices_too_common_values])
    # Freeing up available quantile spaces from values that are too common
    # (have a larger share than permitted by the parameter max_share)
    col_bool_remaining_values = np.ones(d["a"].shape[0], dtype=bool)
    for curr_common_value_idx in indices_too_common_values:
        curr_common_value = col_unique_values[curr_common_value_idx]
        curr_common_value_count = col_value_counts[curr_common_value_idx]
        print("curr_common_value:", curr_common_value, "n_new_avail_quantiles", n_new_avail_quantiles)
        curr_avail_quantile_indices = np.nonzero(
            np.isclose(tr.quantiles_[:, col_idx], curr_common_value)
        )[0][max_n_quantiles_per_value:]
        print("curr_avail_ind", curr_avail_quantile_indices)
        tr.quantiles_[curr_avail_quantile_indices, col_idx] = np.nan
        col_bool_remaining_values *= np.logical_not(np.isclose(d["a"], curr_common_value))
        
    
    # Reset the quantile spaces for values with shares smaller than the parameter max_share, before a redraw
    indices_less_common = np.where(col_percentages <= max_share)
    values_less_common = col_unique_values[indices_less_common]
    for curr_less_common_value in values_less_common:
        print("curr_less_common_value", curr_less_common_value)
        tr.quantiles_[np.nonzero(np.isclose(tr.quantiles_, curr_less_common_value))[0], col_idx] = np.nan
    
    print(tr.quantiles_)
    
    available_quantile_indices = np.where(np.isnan(tr.quantiles_))[0]
    n_available_quantiles = len(available_quantile_indices)
    print("Available quantile indices", available_quantile_indices)
    print("n_available_quantiles:", n_available_quantiles)
    
    print(col_bool_remaining_values)
    col_remaining_values = d["a"].loc[col_bool_remaining_values].to_numpy()
    print(col_remaining_values)
    if col_remaining_values.shape[0] != n_available_quantiles:
        tr.quantiles_[available_quantile_indices, col_idx] = np.nanpercentile(col_remaining_values,
                                                                              q=np.linspace(0, 100, n_available_quantiles))
    else:
        print("equal")
        tr.quantiles_[available_quantile_indices, col_idx] = col_remaining_values
    
    tr.quantiles_[:, col_idx] = np.sort(tr.quantiles_[:, col_idx])

print(tr.quantiles_)